## Preparing Zindi Training data (DEVEX train)

## Project Background

The ultimate goal for my project was to be able tag machine learning datasets belonging to particular Sustainable Development Goals (SDGs). This competition hosted by Zindi provided a really unique dataset, containing text of scraped from various online mediums (articles, grant postings, NGO descriptions, news headlines) all belonging to SDG goal #3 (health and well-being) https://zindi.africa/competitions/sustainable-development-goals-sdgs-text-classification-challenge

This dataset was a great resource for my project since it provided labeled training data I could use to label ML datasets related to SDG goal #3. 

In this notebook, I go through the process of cleaning the training data as would be done for the Zindi competition. The problem posed by Zindi is actually a multi-label classification problem, since each training example can fall under multiple labels representing which specific health and well-being indicator(s) best describe the sample. In later stages of the project, eventually the complexity of the data (# of labels) will be reduced.

## Description of the training data from the Zindi competition website:

The data has been split into a test and training set.

devex_train.csv is the dataset that you will use to train your model. This dataset includes the "outcome" which in this competition is the "label(s)," i.e. which of the 27 indicators are relevant to the given text. This dataset includes the columns ID, Type, Text, and Label_1 through Label_12.


## Thought process

1. Load and clean the data (remove HTML tags)

2. Change list of labels for each sample to one-hot-encoded for all 27 indicators

3. Rearrange the columns and save to csv

## This notebook produces train_CLEAN.csv


## import libraries

In [1]:
#for vectorized operations
import numpy as np
#for dataset visualization and aggregation/organization
import pandas as pd
#for the NLP feature selectors, tfidf, and cleaning text data
import nltk


## Load and clean data

In [2]:
train_df = pd.read_csv("Devex_train.csv", low_memory=False, encoding='latin1')


In [3]:
#the dataset has 2995 samples and 15 columns
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2995 entries, 0 to 2994
Data columns (total 15 columns):
Unique ID    2995 non-null int64
Type         2995 non-null object
Text         2995 non-null object
Label 1      2995 non-null object
Label 2      1635 non-null object
Label 3      738 non-null object
Label 4      312 non-null object
Label 5      142 non-null object
Label 6      59 non-null object
Label 7      21 non-null object
Label 8      10 non-null object
Label 9      4 non-null object
Label 10     2 non-null object
Label 11     0 non-null float64
Label 12     0 non-null float64
dtypes: float64(2), int64(1), object(12)
memory usage: 351.1+ KB


In [4]:
#Text descriptions, this will become part of the training data (1- health related) in later
#stages of the project
train_df.Text

0       Centers of Biomedical Research Excellence (COB...
1       Research on Regenerative Medicine <h2><strong>...
2       Catholic Health Association of India (CHAI): <...
3            Quality Improvement Initiatives for Diabetes
4       Provision of Thalassemia Drugs and Disposables...
                              ...                        
2990     How rats could help reduce the global TB burden:
2991    Exploratory Analyses of Adherence Strategies a...
2992    Study on Vaccines for Diarrhoeal Diseases or L...
2993    Regional Engagement Stimulation Fund on Human ...
2994    Graphic Design Services Consultancy ; ; <p><st...
Name: Text, Length: 2995, dtype: object

## remove html tags

In [6]:
#helper function to remove html nonsense
def cleanhtml(raw_html):
    cleaner = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    #cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner, '', raw_html)
    return cleantext

In [8]:
#nltk contains built in functions for text-cleaning
import nltk

#the download should only have to be run once
#stop words are commonly used words that can be ignored "the", "and", "a", "an" 
nltk.download('stopwords')
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, '', text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = [word for word in text.split() if word not in STOPWORDS]# delete stopwords from text
    text = " ".join(text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/annachesson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
#make copies of the original datasets
clean_train_df = train_df.copy()

In [10]:
#modify the clean versions in relevant column to contain cleaned text
clean_train_df['Text'] = [text_prepare(raw_html) for raw_html in train_df['Text']]

In [15]:
#check the results
print(len(clean_train_df), len(train_df))

2995

In [11]:
#example of text sample after cleaning for html tags and stopwords
clean_train_df['Text'][300]

'clinical trials observational studies behavioral interventions prevention opioid use disorder adjunct medication assisted treatment div classheading4strongbackground strong div pthere opioid crisis united states 4 million people united states report using opioids nonmedical purposes past month almost 2 million report symptoms consistent opioid use disorder oud national survey drug use health nsduh nbsp fewer half oud receive treatment volkow et al 2014 even fewer receive treatment adequate durationnbsp number drug overdose deaths involving opioids quadrupled 1999 2015 33 000 annually rudd et al 2016 p pchronic pain important comorbidity patients oudnbsp twenty 30 percent us adults report chronic pain nahin 2015 treatment acute chronic pain conditions opioids contributing oud epidemic pain patients increased risk developing oud pain inadequately controlled exposed opioids acute pain episodes chronic pain patients history substance abusenbsp among patients oud treatment chronic pain bar

In [12]:
clean_train_df.head

<bound method NDFrame.head of       Unique ID          Type  \
0         12555         Grant   
1         14108         Grant   
2         23168  Organization   
3        219512      Contract   
4        274093        Tender   
...         ...           ...   
2990      92153          News   
2991       1209      Open Opp   
2992      14342         Grant   
2993      12353         Grant   
2994     260848        Tender   

                                                   Text  \
0     centers biomedical research excellence cobre p...   
1     research regenerative medicine h2strongintrodu...   
2     catholic health association india chai pthe ca...   
3              quality improvement initiatives diabetes   
4     provision thalassemia drugs disposables h2stro...   
...                                                 ...   
2990            rats could help reduce global tb burden   
2991  exploratory analyses adherence strategies data...   
2992  study vaccines diarrhoeal diseases l

## Let's make a note of observations about the data so far:

## We are predicting n of 27 labels/indicators
There are 12 label columns in the training set. There are 27 types of labels/indicators. The maximum number of labels assigned to an training sample is 10 labels/indicators.  

## What else may be important to know about the training set? 
1. Distribution of #of labels: I want to know what is the distribution of #of labels (do most examples have only 1-2 labels and only a few have 10?) 

2. Most common labels: I want to know which of the 27 labels is most common/what is the distribution of the occurance of specific labels/indicators

3. One hot encoding: As I mentioned above each sample has several labels (3.1.2, 3.2.3, etc.) associated with it. These labels as is are not are problematic for applying ML models so we will need to binary one-hot-encode each label/ 

To do this we need the data in a better format, we need 27 columns, one for each indicator, with binary entries 0 or 1 to indicate if it was a given label for the example



In [17]:
#the submission file contains all the labels we need to one hot encode
submission_df = pd.read_csv("Devex_submission_format.csv", encoding = "ISO-8859-1")

In [18]:
print(submission_df.columns)
print(submission_df.columns[1:]) #takes a slice after the first column

Index(['ID', '3.1.1', '3.1.2', '3.2.1', '3.2.2', '3.3.1', '3.3.2', '3.3.3',
       '3.3.4', '3.3.5', '3.4.1', '3.4.2', '3.5.1', '3.5.2', '3.6.1', '3.7.1',
       '3.7.2', '3.8.1', '3.8.2', '3.9.1', '3.9.2', '3.9.3', '3.a.1', '3.b.1',
       '3.b.2', '3.b.3', '3.c.1', '3.d.1'],
      dtype='object')
Index(['3.1.1', '3.1.2', '3.2.1', '3.2.2', '3.3.1', '3.3.2', '3.3.3', '3.3.4',
       '3.3.5', '3.4.1', '3.4.2', '3.5.1', '3.5.2', '3.6.1', '3.7.1', '3.7.2',
       '3.8.1', '3.8.2', '3.9.1', '3.9.2', '3.9.3', '3.a.1', '3.b.1', '3.b.2',
       '3.b.3', '3.c.1', '3.d.1'],
      dtype='object')


In [19]:
indicator_labels = submission_df.columns[1:]
indicator_labels

Index(['3.1.1', '3.1.2', '3.2.1', '3.2.2', '3.3.1', '3.3.2', '3.3.3', '3.3.4',
       '3.3.5', '3.4.1', '3.4.2', '3.5.1', '3.5.2', '3.6.1', '3.7.1', '3.7.2',
       '3.8.1', '3.8.2', '3.9.1', '3.9.2', '3.9.3', '3.a.1', '3.b.1', '3.b.2',
       '3.b.3', '3.c.1', '3.d.1'],
      dtype='object')

In [20]:
#drop the columns of label 0-12 which we are going to replace with the indicator labels
clean_train_df = clean_train_df.drop(columns=clean_train_df.columns[3:15])

In [21]:
#now we have all the samples without any label information (yet)
clean_train_df

,Unique ID,Type,Text
0,12555,Grant,centers biomedical research excellence cobre p...
1,14108,Grant,research regenerative medicine h2strongintrodu...
2,23168,Organization,catholic health association india chai pthe ca...
3,219512,Contract,quality improvement initiatives diabetes
4,274093,Tender,provision thalassemia drugs disposables h2stro...
...,...,...,...
2990,92153,News,rats could help reduce global tb burden
2991,1209,Open Opp,exploratory analyses adherence strategies data...
2992,14342,Grant,study vaccines diarrhoeal diseases lower respi...
2993,12353,Grant,regional engagement stimulation fund human imm...


In [22]:
#here we concat our columns of indicator labels to the clean train df from above
clean_train_df = pd.concat([pd.DataFrame(columns=indicator_labels),clean_train_df], sort=False)

In [23]:
#there are no values in the indicator columns yet, so we fill them with 0
clean_train_df.fillna(0, inplace=True)
clean_train_df

,3.1.1,3.1.2,3.2.1,3.2.2,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,3.4.1,...,3.9.3,3.a.1,3.b.1,3.b.2,3.b.3,3.c.1,3.d.1,Unique ID,Type,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12555.0,Grant,centers biomedical research excellence cobre p...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,14108.0,Grant,research regenerative medicine h2strongintrodu...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,23168.0,Organization,catholic health association india chai pthe ca...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,219512.0,Contract,quality improvement initiatives diabetes
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,274093.0,Tender,provision thalassemia drugs disposables h2stro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,92153.0,News,rats could help reduce global tb burden
2991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1209.0,Open Opp,exploratory analyses adherence strategies data...
2992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,14342.0,Grant,study vaccines diarrhoeal diseases lower respi...
2993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12353.0,Grant,regional engagement stimulation fund human imm...


In [24]:
#we can reorganize the columns, #pop takes the column off the df and stores as a series
Unique_id = clean_train_df.pop('Unique ID')
Type = clean_train_df.pop('Type')
Text = clean_train_df.pop('Text')

clean_train_df.insert(0, 'Unique ID', Unique_id)
clean_train_df.insert(1, 'Type', Type)
clean_train_df.insert(2, 'Text', Text)

clean_train_df


,Unique ID,Type,Text,3.1.1,3.1.2,3.2.1,3.2.2,3.3.1,3.3.2,3.3.3,...,3.8.2,3.9.1,3.9.2,3.9.3,3.a.1,3.b.1,3.b.2,3.b.3,3.c.1,3.d.1
0,12555.0,Grant,centers biomedical research excellence cobre p...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14108.0,Grant,research regenerative medicine h2strongintrodu...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23168.0,Organization,catholic health association india chai pthe ca...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,219512.0,Contract,quality improvement initiatives diabetes,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,274093.0,Tender,provision thalassemia drugs disposables h2stro...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,92153.0,News,rats could help reduce global tb burden,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2991,1209.0,Open Opp,exploratory analyses adherence strategies data...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2992,14342.0,Grant,study vaccines diarrhoeal diseases lower respi...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2993,12353.0,Grant,regional engagement stimulation fund human imm...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Refilling the indicator columns
Currently we have the dataset in the format we would like, but we still don't have any information in the indicator columns since they're all filled with 0s! The original labels are stored in the train_df which we first loaded in this notebook. We can iterate through the label columns there, look at the first 5 characters of the label, e.g. len('3.x.x') = 5 and use that to fill in the values on the clean_train_df

In [25]:
#make sure all null values are filled with 0 for comparison in the for loop below
train_df.fillna(0,inplace=True)

#for every row in the training set
for i in range(len(train_df)):
    #for every label column
    for j in range(3,15):
        #if there is anything in this cell
        if train_df.iloc[i,j]!=0:
            label = train_df.iloc[i,j][0:5] #first 5 characters of the string is a label  (e.g. 3.8.1)
            clean_train_df.at[i,label] = 1

In [26]:
#make sure we haven't made any major mistakes!
clean_train_df.shape

(2995, 30)

In [27]:
#save to CSV
clean_train_df.to_csv('train_CLEAN.csv', index=False, encoding='utf-8')

In [28]:
#read back as a new dataframe to inspect
#I have also found this issue when reading a large csv file with the default egine. If I use engine='python' then it works fine.
train_CLEAN = pd.read_csv('train_CLEAN.csv', encoding='utf-8', engine='python')

In [29]:
train_CLEAN.shape

(2995, 30)

In [49]:
train_CLEAN.columns

Index(['Unique ID', 'Type', 'Text', '3.1.1', '3.1.2', '3.2.1', '3.2.2',
       '3.3.1', '3.3.2', '3.3.3', '3.3.4', '3.3.5', '3.4.1', '3.4.2', '3.5.1',
       '3.5.2', '3.6.1', '3.7.1', '3.7.2', '3.8.1', '3.8.2', '3.9.1', '3.9.2',
       '3.9.3', '3.a.1', '3.b.1', '3.b.2', '3.b.3', '3.c.1', '3.d.1'],
      dtype='object')

## and done! 

A breif summary of what was accomplished: 

We used the information about the labels in the submission_format csv to reconstruct the training set from lists of labels, to one hot encoded for all 27 possible labels.

The result of this notebook is train_CLEAN.csv